In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
from scipy import sparse 

In [3]:
def Shat(A,T,G): 
    
    t = np.tanh(T/4 + G/4)
    tt = np.tanh(T/4 - G/4)
    Shatf = np.array([[1-t*tt,0,0,t+tt],[0,1+t*tt,t-tt,0],[0,-t+tt,1+t*tt,0],[-t-tt,0,0,1-t*tt]],dtype=complex)
    Shatg = np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,-1]],dtype=complex)
    f = -(1j/4)*A*(2*np.cosh(T/2)+2*np.cosh(G/2))*(1/np.sinh(T/2))*(1/np.cosh(T/2))
    Shat = f*Shatf+Shatg
    Nalpha = 1+2*A**(2)*(np.cosh(T)/np.sinh(T))*(1/np.sinh(T))+2*A**(2)*np.cosh(G)*(1/np.sinh(T))**(2)
    Shat = (-1/np.sqrt(Nalpha))*Shat
    return Shat

def dense_to_sparse(dense_matrix):
    # Convert the dense matrix to a sparse matrix in CSR format
    sparse_matrix = sparse.csr_matrix(dense_matrix)
    return sparse_matrix

def sparse_perm_nsite(U,n):   #CHECKED#
    if n < 2:
        raise ValueError("Number of sites (n) must be at least 2.")
    
    #Constructing a sparse array using ijv format
    row  = np.array([0, 1, 2, 3])
    col  = np.array([0, 2, 1, 3])
    data = np.array([1, 1, 1, -1])
    P=sparse.coo_array((data, (row, col)), shape=(4, 4)).toarray()

    perm_list=[]
    # Construct permutation matrices and tensor products
    for i in range(0, n - 1):
        Pi = sparse.kron(sparse.eye(2 ** i,format='csr'), sparse.kron(P, sparse.eye(2 ** (n - i - 2),format='csr')))
        perm_list.append(Pi)
        
    # Append U tensor product
    S_last = sparse.kron(sparse.eye(2 ** (n - 2),format='csr'), U)
    perm_list.append(S_last)

    result = sparse.eye(2**n,format='csr')
    #print(result, result.shape)
    for perm_matrix in perm_list:
        result = sparse.csr_matrix.dot(result, perm_matrix)

    perm_list.reverse()

    for perm_matrix in perm_list[1:]:
        result = sparse.csr_matrix.dot(result,  perm_matrix)    
        
    return result

def sp_bw(mat,n):
    list=[]
    matrix=dense_to_sparse(mat)
    matrixZ=sparse_perm_nsite(matrix,n)

    #even layer
    for i in range(0, n//2):
        U = sparse.kron(sparse.eye(2 ** (2*i),format='csr'), sparse.kron(matrix, sparse.eye(2 ** (n - 2*i - 2),format='csr')))
        #print(i,'appending even',U.shape)
        list.append(U)
    #odd layer
    for i in range(0, n//2):
        #print(i)
        if i == (n//2 - 1):
            #print('appending ZZ',matrixZ.shape)
            #print('swag')
            list.append(matrixZ)
        else: 
            U = sparse.kron(sparse.eye(2 ** (2*i+1),format='csr'), sparse.kron(matrix, sparse.eye(2 ** (n - 2*i - 3),format='csr')))
            #print('appending odd',U.shape)
            list.append(U)
    

    #INITIAL STATE
    temp = np.zeros((2**n))
    temp[0]=1
    
    print(len(list))
    for j,elem in enumerate(list):
        print(j+1)
        temp = sparse.csr_matrix.dot(elem, temp)
        

    return temp


In [5]:
n=20; mat = Shat(1,1,1)
op=sp_bw(mat,n)
print(op.shape,op)


20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
(1048576,) [-2.08192763e-01-3.84966816e-02j  0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j ...  0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j -3.38813179e-21+5.63441547e-05j]


We are now able, given an input string, to compute t-evol after one UBW (even+odd) up to 22 sites
Next things to implement:
    - ability to apply single layers (l=3 means ev odd ev) and not 3 full UBWS 
    - measurement protocol 
    - averaging over many realisations per layer (shots routine)